# Odovzdanie 2 - Predspracovanie
#### Anton Rusňák, František Gič

In [1290]:
import pandas as pd
import numpy as np
import matplotlib as mat
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import seaborn as sns
import statsmodels.api as sm
import statsmodels.stats as sm_stats
import statsmodels.stats.api as sms
import scipy.stats as stats
import statistics 
from statistics import mode 
%matplotlib inline

Nacitanie pomocnych funkcii zo skriptu

In [1291]:
%run "functions.py"

### Nacitanie datasetov 

In [1292]:
data_personal = load("data/personal_train.csv")
data_other = load("data/other_train.csv")

print(len(data_personal))
print(len(data_other))

3933
3983


### Spajanie datasetov

In [1293]:
data = pd.merge(data_personal,data_other,on=['name','address'], how = 'outer')

print(len(data))

3983


### Predspracovanie jednotlivych atributov

#### Atribut sex

Neobsahuje ziadne prazdne hodnoty, len stripneme whitespace a nahradime kategoricke hodnoty ich ciselnymi reprezentaciami - 1 (Male) a 0 (Female)

In [1294]:
print(len((data[data.sex.isnull()])))

0


In [1295]:
def sanitize_sex(sex):
    return 1 if sex.strip() == 'Male' else 0

In [1296]:
data.sex = data.sex.map(lambda sex: sanitize_sex(sex))

In [1297]:
data.sex.unique()

array([1, 0])

#### Atribut age<br>
Ako aj v prvom odovzdani spracuvame age na ciselny atribut a jeho zaporne hodnoty nahradime NaN

In [1298]:
data.age = data.age.map(sanitize_age)

In [1299]:
len(data[pd.isnull(data.age)])

756

Po spracovani vsak mame vela prazdnych hodnot (756).<br>
Napada nas sposob vypocet nahradenim podla datumu, ale nemame zatial rozparsovany atribut date of birth.
<br>

Mohli by sme nahradit chybajuce hodnoty pacienta priemerom veku jeho pohlavia.
Pokial by pacient nemal uvedene pohlavie, nahradili by sme primerom vsetkych pacientov. Tuto funkciu si naprogramujeme, a uvidime ci sa bude dat pouzit neskor.

In [1300]:
def fill_null_age(data): 
	unique_sexes = data.sex.unique()
	for sex in unique_sexes:
		mean = data.loc[(data.age.notnull()) & (data.sex == sex)].age.mean()
		data.loc[(data.sex == sex) & (data.age.isna()),'age'] = mean
	return data	

#### Atribut pregnant<br>
Upravime boolean hodnoty a nahradime ich ciselnou reprezentaciou.
<br>Taktiez vsetkym muzom nastavime pregnant na 0

In [1301]:
def sanitize_boolean(boolean):
    try:
        if boolean.strip() in ['f','F','FALSE','false','False']:
            return 0
        elif boolean.strip() in ['t','T','TRUE','true','True']:
            return 1
        else:
            return np.nan
    except AttributeError:
        return np.nan

In [1302]:
data.pregnant = data.pregnant.map(sanitize_boolean)

In [1303]:
def sanitize_pregnancy(data):
	data.loc[(data.sex == 1),'pregnant'] = 0
	return data

data = sanitize_pregnancy(data)	

Po vycisteni atributu pregnant by sme mohli v pripade ak nejaky zaznam ma chybajuci atribut sex a zaroven ma pregnant 1 -  nastavit ako sex ako zenu (0).
V tomto datasete take nemame, ale je dobre pripravit nasu funkciu just in case.

In [1304]:
def sanitize_pregnancy(data):
	data.loc[(data.sex == 1),'pregnant'] = 0
	data.loc[(data.sex.isnull()) & (data.pregnant == 1),'sex'] = 0
	return data

data = sanitize_pregnancy(data)	

Vidime ze v datasete mame aj prazdne hodnoty, budeme sa tym zaoberat neskor

In [1305]:
print(data.pregnant.unique())
data.pregnant.isnull().sum()

[ 0.  1. nan]


1

#### Atribut personal info<br>
Vyparsujeme vsetky hodnoty z tohto stringu a vytvorime nove columny

In [1306]:
parsed_info = data.personal_info.map(parse_personal_info).map(remove_empty)

In [1307]:
personal_info_columns = ['employment','country','relationship_info','employment_info','race']
for index,name in enumerate(personal_info_columns):
    data[name] = parsed_info.map(lambda x: fill_value(index,x))
	
data = data.drop('personal_info', axis=1)

In [1308]:
data[personal_info_columns].head(5)

,employment,country,relationship_info,employment_info,race
0,Transport-moving,United-States,Divorced,Private,White
1,Adm-clerical,United-States,Divorced,Private,White
2,Prof-specialty,United-States,Married-civ-spouse,Private,White
3,Prof-specialty,United-States,Married-civ-spouse,State-gov,White
4,Tech-support,United-States,Never-married,Private,White


#### Atribut date of birth<br>
Z prvotnej analyzy vieme ze date of birth nema unifikovany tvar datumu, cize bude nutne ho osetrit.

In [1309]:
def sanitize_date(date):
    date = str(date).replace('/', '-')
    date = date[:10]
    date = date.split("-")
    
    if date[0] != 'nan':
        if len(date[0]) != 4:
            if len(date[2]) == 2 and int(date[0]) > 31:
                new_date = "19"+ date[0] +"-"+date[1]+"-"+date[2] 
                
            elif ((len(date[2]) == 2) and (int(date[0]) < 31) and (int(date[2]) > 31)):
                new_date = "19"+date[2]+"-"+date[1]+"-"+date[0] 
                
            elif ((len(date[2]) == 2) and (int(date[0]) < 31) and (int(date[2]) < 31)):
                new_date = "20"+ date[2] + "-" +date[1]+"-" + date[0] 
            else:
                new_date = date[2]+"-"+date[1]+"-"+date[0] 
            return new_date        
    return '-'.join(date)

In [1310]:
data.date_of_birth = data.date_of_birth.map(sanitize_date)
data.date_of_birth.head(5)

0    1960-07-04
1    1972-07-20
2    1960-02-28
3    1967-12-04
4    1938-04-22
Name: date_of_birth, dtype: object

Vidime, ze datumy maju uz rovnaky format. Teraz sa mozme vratit k upravovaniu atributu age - vypocitat chybajuce hodnoty na zaklade datumu.

### Dopocitanie veku na zaklade datumu

Prejdeme vsetky datumy a najdeme najfrekventovanejsi rok po scitani veku a roku narodenia. To je rok, v ktorom bolo uskutocnene meranie (supis) veku.
Tento rok vyuzijeme na dopocitanie veku osob na zaklade ich datumu narodenia ktore ho nemaju.

In [1311]:
years = [];

def store_measure_year(age,date):
    if (pd.notnull(age)):
        years.append(int(date.split('-')[0]) + age)
        
data.apply(lambda x: store_measure_year(x.age,x.date_of_birth),axis=1)

measure_year = pd.Series(years).mode();

In [1312]:
def fill_null_age(age,date):
    if (date):
        return (measure_year - int(date.split('-')[0]))[0]
    return np.nan

data.loc[data.age.isnull(),'age'] = data[data.age.isnull()].apply(lambda x: fill_null_age(x.age,x.date_of_birth),axis=1)

Podarilo sa nam vsetkym zaznamom dopocitat vek.

In [1313]:
len(data[data.age.isnull()])

0

Nastastie nemame ziadne take ze bez datumu a bez veku.

In [1314]:
len(data[(data.date_of_birth.isna()) & (data.age.isnull())])

0

#### Atribut income
Prehodime kategoricke hodnoty na ciselnu reprezentaciu ('<=50K' == 0, '>50K' == 1)

In [1315]:
data.income.unique()

array([' <=50K', ' >50K', nan], dtype=object)

In [1316]:
data.income = data.income.map(lambda income: 0 if str(income).strip() == '<=50K' else 1 if str(income).strip() == '>50K' else np.nan)

In [1317]:
data.income.unique()

array([ 0.,  1., nan])

#### String Atributy
Tieto atributy su stringove, viackrat duplicitne, skusime ich troska quickfixnut a zjednotit data

In [1318]:
def strip_string(string):
    try:
        return str(string).strip()
    except AttributeError:
        return None

def sanitize_string(string):
    string = strip_string(string);
    if string in ['None','nan','??','?']:
        return None
    return string

def sanitize_multiword_string(string):
    try:
        return sanitize_string(string).replace('-',' ').replace('_',' ')
    except AttributeError:
        return None

for column in ['relationship','relationship_info','employment','employment_info']:
    data[column] = data[column].map(sanitize_multiword_string)
    print(column,': ',data[column].unique(),'\n')


relationship :  ['Not in family' 'Own child' 'Husband' 'Wife' 'Unmarried' 'Other relative'
 None] 

relationship_info :  ['Divorced' 'Married civ spouse' 'Never married' None 'Widowed'
 'Married spouse absent' 'Separated' 'Married AF spouse'] 

employment :  ['Transport moving' 'Adm clerical' 'Prof specialty' 'Tech support' None
 'Sales' 'Craft repair' 'Other service' 'Handlers cleaners'
 'Farming fishing' 'Exec managerial' 'Machine op inspct' 'Priv house serv'
 'Protective serv' 'Armed Forces'] 

employment_info :  ['Private' 'State gov' None 'Self emp not inc' 'Federal gov' 'Local gov'
 'Self emp inc' 'Without pay' 'Never worked'] 



In [1319]:
print(len(data[data.relationship.isnull()]))
print(len(data[data.relationship_info.isnull()]))

13
12


Velmi ma zaujali atributy relationship a relationship_info, a rad by som doplnil ich nejake prazdne hodnoty. 

Najskor by som asi zjednotil hodnoty 'Married civ spouse','Married AF spouse','Married spouse absent', na 'Married', vsak to je asi jedno za koho je zenaty/vydata. Teda vlasne iba ten treti typ to vyhral, ten ma o problem postarane.<br>
<img src="http://m.quickmeme.com/img/db/dbd8d8db8056e48ac72fd40435157978a1bbf2b63b297b41a9ba4302ed9962e0.jpg" height=200 width=200 align="left">

In [1320]:
data.loc[data.relationship_info.str.contains('Married',na=False),'relationship_info'] = data[data.relationship_info.str.contains('Married',na=False)].relationship_info.map(lambda x: 'Married')

Dalsie co mozeme urobit je ze ak ma dany clovek relationship_info Never married, tak relationship mozme nastavit na Unmarried.

In [1321]:
def sanitize_relationship(relationship,relationship_info):
    if relationship_info == 'Never married':
            return 'Unmarried'
    return relationship
    
data.loc[data.relationship.isnull(),'relationship'] = data[data.relationship.isnull()].apply(lambda row: sanitize_relationship(row.relationship,row.relationship_info), axis=1)

Este jedna vec nas napada, a to ti, ktori maju Husband alebo Wife v relationship, tak im implicitne dame Married.

In [1322]:
def sanitize_relationship_info(relationship,relationship_info):
    if relationship in ['Wife','Husband']:
            return 'Married'
    return relationship_info
    
data.loc[data.relationship_info.isnull(),'relationship_info'] = data[data.relationship_info.isnull()].apply(lambda row: sanitize_relationship_info(row.relationship,row.relationship_info), axis=1)

Mozno este ti co maju relationship Married tak sa pozriet na ich pohlavie a podla toho dat Wife alebo Husband? (No nechceme to hrotit, takze to nehame tak)

In [1323]:
print(data[data.relationship.isnull()].relationship_info,'\n')
print(data[data.relationship_info.isnull()].relationship)

1132        None
1442     Married
1516    Divorced
1837        None
2018        None
3161     Married
3162     Married
3449     Married
3801     Married
3963     Married
Name: relationship_info, dtype: object 

1009    Not in family
1132             None
1535    Not in family
1837             None
2018             None
2506        Unmarried
Name: relationship, dtype: object


Pozrieme sa este na country a education:

In [1324]:
for column in ['country','education','race']:
    data[column] = data[column].map(sanitize_string)
    print(column,': ',data[column].unique(),'\n')

country :  ['United-States' None 'El-Salvador' 'South' 'Germany' 'Mexico' 'Cuba'
 'Philippines' 'England' 'Ireland' 'Canada' 'China' 'France' 'Japan'
 'India' 'Scotland' 'Italy' 'Puerto-Rico' 'Portugal' 'Jamaica' 'Thailand'
 'Dominican-Republic' 'Nicaragua' 'Poland' 'Peru' 'Vietnam' 'Ecuador'
 'Iran' 'Columbia' 'Laos' 'Guatemala' 'Hong' 'Haiti'
 'Outlying-US(Guam-USVI-etc)' 'Honduras' 'Cambodia' 'Taiwan' 'Greece'
 'Trinadad&Tobago' 'Yugoslavia'] 

education :  ['HS-grad' 'Bachelors' 'Some-college' '10th' 'Assoc-acdm' 'Masters' None
 '1st-4th' '7th-8th' 'Prof-school' '11th' '5th-6th' 'Assoc-voc' '9th'
 'Doctorate' '12th' 'Preschool'] 

race :  ['White' None 'Asian-Pac-Islander' 'Black' 'Other' 'Amer-Indian-Eskimo'] 



### Odstranenie duplikatov

Rozhodli sme sa odstranit duplikaty na zaklade mena, adresy a datumu narodenia.

In [1326]:
print('length other:',len(data_other))
print('length personal',len(data_personal))

length other: 3983
length personal 3933


In [1327]:
print('length data',len(data))

length data 3983


In [1330]:
print('length duplicated before: ',len(data[data.duplicated(['name','address','date_of_birth'])]))
data = data.drop_duplicates(['name','address','date_of_birth'],keep="last")
print('length data',len(data))
print('length duplicated after: ',len(data[data.duplicated(['name','address','date_of_birth'])]))

length duplicated before:  50
length data 3933
length duplicated after:  0


# ------------------------------------------------------
# Kamosko tolko zatial odomna asi

### Chybajuce hodnoty

In [708]:
data[data.columns[data.isnull().any()].tolist()].isnull().sum()

kurtosis_oxygen       5
pregnant              1
education-num        18
relationship         13
skewness_glucose     23
mean_glucose         10
std_oxygen           14
capital-gain         16
skewness_oxygen      13
kurtosis_glucose     18
education            12
fnlwgt               13
class                19
std_glucose          12
income               14
mean_oxygen          16
hours-per-week       13
capital-loss         20
employment           12
country              12
relationship_info    12
employment_info      12
race                 12
dtype: int64

### Predikcia pregnant

In [552]:
from sklearn.preprocessing import Imputer
from sklearn import linear_model

imp_median = Imputer(missing_values='NaN', strategy='median')

/Users/feri/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:66: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


In [553]:
data['sex'] = imp_median.fit_transform(data[['sex']])
data['mean_glucose'] = imp_median.fit_transform(data[['mean_glucose']])
data[['pregnant','sex','mean_glucose']].astype('float64').corr()
data.mean_glucose.astype('float64').isnull().any()
data['sex'].isnull().any()

False

In [556]:
fin = data.pregnant.dropna()
fin

0       0
1       0
2       0
3       0
4       0
5       0
6       0
7       0
8       0
9       0
10      0
11      0
12      0
13      0
14      0
15      0
16      0
17      0
18      0
19      0
20      0
21      0
22      0
23      0
24      0
25      0
26      0
27      0
28      0
29      0
       ..
3953    0
3954    0
3955    0
3956    0
3957    0
3958    0
3959    0
3960    0
3961    0
3962    0
3963    0
3964    0
3965    0
3966    0
3967    0
3968    0
3969    0
3970    0
3971    0
3972    0
3973    0
3974    0
3975    0
3976    0
3977    0
3978    0
3979    0
3980    0
3981    0
3982    0
Name: pregnant, Length: 3982, dtype: object

In [557]:
X = data[['sex','mean_glucose']].head(fin.count())
y = fin
lm = linear_model.LinearRegression()
model = lm.fit(X,y)
print(lm.score(X,y), 'Úspešnosť doplnenia')
missingTT4 = data[['sex','mean_glucose']].loc[data['pregnant'].isna()]
predictions = lm.predict(missingTT4)
predictions

0.003919711803099357 Úspešnosť doplnenia


array([0.02406011])

In [558]:
missing = data.pregnant.isna()
onlyNans = data[missing]
data.loc[onlyNans.pregnant.index,'pregnant'] = predictions

In [561]:
data[data.name.str.contains('Joshua Sanders')]

,name,address,age,sex,date_of_birth,kurtosis_oxygen,pregnant,education-num,relationship,skewness_glucose,...,std_glucose,income,mean_oxygen,hours-per-week,capital-loss,employment,country,relationship_info,employment_info,race
2002,Joshua Sanders,0629 Trevor Inlet Suite 858\r\nHarrisonborough...,37.0,0.0,1982-05-19,11.938502,0,6.0,Unmarried,-0.368959,...,55.480443,<=50K,1.644649,40.0,0.0,Sales,United-States,Divorced,Private,White
2003,Joshua Sanders,0629 Trevor Inlet Suite 858\r\nHarrisonborough...,37.0,0.0,1982-05-19,11.938502,0.0240601,NaN,Unmarried,-0.368959,...,55.480443,<=50K,1.644649,40.0,0.0,Sales,United-States,Divorced,Private,White
